### Importações de bibliotecas

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
# from pycaret.classification import *
# import random
# import pickle
import numpy as np
# import statsmodels.stats.proportion as sm
# from scipy import stats
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# import matplotlib.pyplot as plt
# import math

### Informações do método e funções

In [2]:
# Define a variável target
target = 'Back_Over'

# Cria classes do target
def cria_alvos(df):
    # Back Home
    df.loc[(df['Home_Pts'] > df['Away_Pts']), 'Back_Home'] = 1
    df.loc[(df['Home_Pts'] < df['Away_Pts']), 'Back_Home'] = 0
    
    df.loc[(df['Back_Home']) == 1, 'PL_Home'] = df.Odds_H - 1
    df.loc[(df['Back_Home']) == 0, 'PL_Home'] = - 1
    
    # Back Away
    df.loc[(df['Home_Pts'] < df['Away_Pts']), 'Back_Away'] = 1
    df.loc[(df['Home_Pts'] > df['Away_Pts']), 'Back_Away'] = 0
    
    df.loc[(df['Back_Away']) == 1, 'PL_Home'] = df.Odds_A - 1
    df.loc[(df['Back_Away']) == 0, 'PL_Home'] = - 1
    
    # Over/Under
    df.loc[(df['Home_Pts'] + df['Away_Pts']) > df['Over_Line'], 'Back_Over'] = 1
    df.loc[(df['Home_Pts'] + df['Away_Pts']) < df['Over_Line'], 'Back_Over'] = 0
    
    df.loc[(df['Back_Over']) == 1, 'PL_Over'] = df.Odds_Over - 1
    df.loc[(df['Back_Over']) == 0, 'PL_Over'] = - 1

    df.loc[(df['Back_Over']) == 0, 'PL_Under'] = df.Odds_Under - 1
    df.loc[(df['Back_Over']) == 1, 'PL_Under'] = - 1
    
    # HA
    df.loc[((df['Home_Pts'] + df['HA_Line'])) > df['Away_Pts'], 'Back_HA_H'] = 1
    df.loc[((df['Home_Pts'] + df['HA_Line'])) < df['Away_Pts'], 'Back_HA_H'] = 0
    df.loc[((df['Home_Pts'] + df['HA_Line'])) == df['Away_Pts'], 'Back_HA_H'] = 2
    
    df.loc[(df['Back_HA_H']) == 1, 'PL_HA_H'] = df.HA_Odds_H - 1
    df.loc[(df['Back_HA_H']) == 0, 'PL_HA_H'] = - 1
    df.loc[(df['Back_HA_H']) == 2, 'PL_HA_H'] = - 0

    df.loc[(df['Back_HA_H']) == 0, 'PL_HA_A'] = df.HA_Odds_A - 1
    df.loc[(df['Back_HA_H']) == 1, 'PL_HA_A'] = - 1
    df.loc[(df['Back_HA_H']) == 2, 'PL_HA_A'] = - 0

    return df

### Funções

In [3]:
# Calcula o profit
def calcula_profit(df):
    stake = 1
    green = stake * (df.Odds_Over - 1)
    red = -stake

    df.loc[(df['prediction_label'] == 1) & (df[target] == 1), 'Profit'] = green
    df.loc[(df['prediction_label'] == 1) & (df[target] == 0), 'Profit'] = red
    df.loc[(df['prediction_label'] != 1),'Profit'] = 0

    return df

In [4]:
# Calcula o winrate
def calcula_wr(df):

    certos = df[(df['prediction_label'] == 1) & (df[target] == 1)]
    winrate = len(certos) / len(df)
    winrate = round(winrate*100, 2)

    return winrate

In [5]:
def grafico_por_mes(df):
    # Construindo o gráfico de profit acumulado
    df['Date'] = pd.to_datetime(df['Date'])
    df['mes'] = df['Date'].dt.month
    df['Profit_acu'] = df['Profit'].cumsum()
    ax = df.groupby('mes')['Profit_acu'].last().plot(kind='line', figsize=(5,3), marker='o')
    ax.set_xlabel('Mês')
    ax.set_ylabel('Profit Acumulado')
    ax.set_xticks(df['mes'].unique())
    for i,j in zip(df['mes'].unique(),df.groupby('mes')['Profit_acu'].last()):
        ax.annotate('{:.2f}'.format(j),xy=(i,j), fontsize=9)

    plt.show()

In [6]:
def plota_grafico(df):
    df.Profit_acu.plot(figsize=(6,5), marker='o')
    last_x = df['Profit_acu'].index[-1]
    last_y = df['Profit_acu'].values[-1]
    plt.text(last_x, last_y, str(round(last_y, 2)))
    plt.show()

In [7]:
def risco_ruina(df, stake=1, perc_banca=1, num_blocos=5):
     dados = df.Profit.tolist()
     
     if stake != 1:
          multiplicado = []
          for valor in dados:
               multiplicado.append(valor * stake)
          dados = multiplicado

     blocos = np.array_split(dados, num_blocos)

     medias = []

     for bloco in blocos:
          meds = np.mean(bloco)
          medias.append(meds)

     desv_pad = np.std(medias)
     media_geral = np.mean(medias)

     # Risco de ruína
     risco = math.exp((-2 * media_geral * 1) / (desv_pad ** 2)) * 100

     # Intervalo de confiança
     margem_erro = 1.96 * (desv_pad/(num_blocos ** (1/2)))
     margem_cima = round((media_geral + margem_erro), 2)
     margem_baixo = round((media_geral - margem_erro), 2)
     
     media_por_dp = media_geral / desv_pad
     raiz = 1.96 / (num_blocos ** (1/2))

     print(f'Média: {round(media_geral,2)}')
     print(f'Desvio padrão: {round(desv_pad,2)}')
     print('')
     print(f'Blocos de {len(bloco)} partidas')
     print(f'Risco de ruína: {risco:.2f}%')
     print('')
     print(f"Intervalo de confiança: ({margem_baixo}, {margem_cima})")
     print('')
     print(f'Média/DP: {media_por_dp:.2f}')
     print(f'1,96/raiz: {raiz:.2f}')

In [8]:
def int_confianca(df):
    dados = df.Profit.tolist()

    # Nível de confiança desejado (por exemplo, 95%)
    confianca = 0.95

    # Cálculo do intervalo de confiança para a média
    media = np.mean(dados)
    n = len(dados)
    erro_padrao = stats.sem(dados)
    intervalo = stats.t.interval(confianca, df=n-1, loc=media, scale=erro_padrao)
    intervalo = tuple(round(item, 2) for item in intervalo)

    # Exibição do resultado
    print(f'Média: {round(media,2)}')
    print(f'Desvio padrão: {round(erro_padrao,2)}')
    print(f"Intervalo de confiança: {intervalo}")

In [9]:
def teste_variancia(df):
    dados = df.Profit.tolist()

    # Cálculo do intervalo de confiança para a média
    media = np.mean(dados)
    n = len(dados)
    erro_padrao = stats.sem(dados)
    desvio_padrao = np.std(dados)

    media_por_dp = media / desvio_padrao
    raiz = 1.96 / math.sqrt(n)

    # Exibição do resultado
    print(f'Média/DP: {media_por_dp:.2f}')
    print(f'1,96/raiz: {raiz:.2f}')

In [10]:
################################################################################
# Prepara o DF
################################################################################

def prepara_df(df):
  df = df[(df.Over_Line >= 5)]
  df = df[(df['Home_Pts'] + df['Away_Pts']) != df['Over_Line']]
  df = df[df['HA_Odds_A'] != 0]
  df = df[df['Odds_H'] != 0]
  df = df[df['Odds_A'] != 0]
  df = df[df['Odds_Over'] != 0]
  df = df[df['Odds_Under'] != 0]

  df = cria_alvos(df)

  df['P(H)'] = 1 / df['Odds_H']
  df['P(A)'] = 1 / df['Odds_A']
  df['P(O)'] = 1 / df['Odds_Over']
  df['P(U)'] = 1 / df['Odds_Under']
  
  df['P_Diff'] = ((1 / df['Odds_H']) + (1 / df['Odds_A'])) - 1
  df['Porc_Over_Home'] = df.groupby('Home')[target].rolling(3).mean().reset_index(level=0, drop=True)
  df['Porc_Over_Away'] = df.groupby('Away')[target].rolling(3).mean().reset_index(level=0, drop=True)
  df['Porc_Over_Home']  = df.groupby('Home')['Porc_Over_Home'].shift(1)
  df['Porc_Over_Away']  = df.groupby('Away')['Porc_Over_Away'].shift(1)
  df['Porc_Over_Home'] = df['Porc_Over_Home'].replace(np.nan, 0)
  df['Porc_Over_Away'] = df['Porc_Over_Away'].replace(np.nan, 0)
  df.reset_index(inplace=True, drop=True)

  # Custo do gol
  df['CustoGolHome'] = df['Home_Pts'] / (1 / df['Odds_H'])
  df['CustoGolAway'] = df['Away_Pts'] / (1 / df['Odds_A'])
  df['CustoGolHome'] = df['CustoGolHome'].replace(np.inf, 0)
  df['CustoGolAway'] = df['CustoGolAway'].replace(np.inf, 0)
  df.reset_index(drop=True, inplace=True)
  
  # Último custo do gol
  df['Last_CG_H']  = df.groupby('Home')['CustoGolHome'].shift(1)
  df['Last_CG_A']  = df.groupby('Away')['CustoGolAway'].shift(1)
  df['Last_CG_H'] = df['Last_CG_H'].replace(np.nan, 0)
  df['Last_CG_A'] = df['Last_CG_A'].replace(np.nan, 0)

  # Média móvel do custo do gol
  df['MediaCustoGolHome'] = df.groupby('Home')['CustoGolHome'].rolling(window=3).mean().reset_index(level=0, drop=True)
  df['MediaCustoGolAway'] = df.groupby('Away')['CustoGolAway'].rolling(window=3).mean().reset_index(level=0, drop=True)

  df['MediaCustoGolHome']  = df.groupby('Home')['MediaCustoGolHome'].shift(1)
  df['MediaCustoGolAway']  = df.groupby('Away')['MediaCustoGolAway'].shift(1)

  df['MediaCustoGolHome'] = df['MediaCustoGolHome'].replace(np.nan, 0)
  df['MediaCustoGolAway'] = df['MediaCustoGolAway'].replace(np.nan, 0)

  limit_up_h = df.CustoGolHome.mean() + df.CustoGolHome.std()
  limit_up_a = df.CustoGolAway.mean() + df.CustoGolAway.std()
  df.loc[(df['CustoGolHome'] > limit_up_h), 'Acima_Last_CG_H'] = 1
  df.loc[(df['CustoGolHome'] <= limit_up_h), 'Acima_Last_CG_H'] = 0
  df.loc[(df['CustoGolAway'] > limit_up_a), 'Acima_Last_CG_A'] = 1
  df.loc[(df['CustoGolAway'] <= limit_up_a), 'Acima_Last_CG_A'] = 0
  df['Acima_Last_CG_H']  = df.groupby('Home')['Acima_Last_CG_H'].shift(1)
  df['Acima_Last_CG_A']  = df.groupby('Away')['Acima_Last_CG_A'].shift(1)
  df['Acima_Last_CG_H'] = df['Acima_Last_CG_H'].replace(np.nan, 0)
  df['Acima_Last_CG_A'] = df['Acima_Last_CG_A'].replace(np.nan, 0)

  limit_down_h = df.CustoGolHome.mean() - df.CustoGolHome.std()
  limit_down_a = df.CustoGolAway.mean() - df.CustoGolAway.std()
  df.loc[(df['CustoGolHome'] < limit_down_h), 'Abaixo_Last_CG_H'] = 1
  df.loc[(df['CustoGolHome'] >= limit_down_h), 'Abaixo_Last_CG_H'] = 0
  df.loc[(df['CustoGolAway'] < limit_down_a), 'Abaixo_Last_CG_A'] = 1
  df.loc[(df['CustoGolAway'] >= limit_down_a), 'Abaixo_Last_CG_A'] = 0
  df['Abaixo_Last_CG_H']  = df.groupby('Home')['Abaixo_Last_CG_H'].shift(1)
  df['Abaixo_Last_CG_A']  = df.groupby('Away')['Abaixo_Last_CG_A'].shift(1)
  df['Abaixo_Last_CG_H'] = df['Abaixo_Last_CG_H'].replace(np.nan, 0)
  df['Abaixo_Last_CG_A'] = df['Abaixo_Last_CG_A'].replace(np.nan, 0)

  df['CV_ML'] = (df[['Odds_H', 'Odds_A']].std(axis=1)) / (df[['Odds_H', 'Odds_A']].mean(axis=1))
  df['CV_Over'] = (df[['Odds_Over', 'Odds_Under']].std(axis=1)) / (df[['Odds_Over', 'Odds_Under']].mean(axis=1))

  df.pop('CustoGolHome')
  df.pop('CustoGolAway')
  
  return df

### Ajuste dos datasets de treino e teste

In [11]:
treino0 = pd.read_csv('../data/kbo/kbo-2019.csv')
treino1 = pd.read_csv('../data/kbo/kbo-2020.csv')
treino2 = pd.read_csv('../data/kbo/kbo-2021.csv')
treino3 = pd.read_csv('../data/kbo/kbo-2022.csv')
treino4 = pd.read_csv('../data/kbo/kbo-2023.csv')
treino = pd.concat([treino0, treino1, treino2, treino3, treino4])
treino.sort_values('Date', inplace=True)
flt = (treino['Home_Pts'] != treino['Away_Pts'])
treino = treino[flt]
treino.reset_index(inplace=True)

treino = prepara_df(treino)
treino.drop('index', axis=1, inplace=True)

colunas_float = ['Home_Pts', 'Away_Pts',
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Back_Home', 'PL_Home', 'PL_Over', 'PL_Under', 'P(H)', 'P(A)', 'P(O)',
       'P(U)', 'P_Diff', 'Porc_Over_Home', 'Porc_Over_Away', 'MediaCustoGolHome', 'MediaCustoGolAway', 'Last_CG_H', 'Last_CG_A', 'CV_ML', 'CV_Over']
treino[colunas_float] = treino[colunas_float].astype(float)

In [18]:
treino.columns

Index(['Date', 'Season', 'Season_Time', 'Time', 'Home', 'Away', 'Home_Pts',
       'Away_Pts', 'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under',
       'HA_Line', 'HA_Odds_H', 'HA_Odds_A', 'Back_Home', 'PL_Home',
       'Back_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', 'P(H)', 'P(A)', 'P(O)', 'P(U)', 'P_Diff', 'Porc_Over_Home',
       'Porc_Over_Away', 'Last_CG_H', 'Last_CG_A', 'MediaCustoGolHome',
       'MediaCustoGolAway', 'Acima_Last_CG_H', 'Acima_Last_CG_A',
       'Abaixo_Last_CG_H', 'Abaixo_Last_CG_A', 'CV_ML', 'CV_Over'],
      dtype='object')

In [19]:
cor = treino[['Back_Over', 'P(H)', 'P(A)', 'P(O)', 'P(U)', 'P_Diff', 'Porc_Over_Home',
       'Porc_Over_Away', 'Last_CG_H', 'Last_CG_A', 'MediaCustoGolHome',
       'MediaCustoGolAway', 'Acima_Last_CG_H', 'Acima_Last_CG_A',
       'Abaixo_Last_CG_H', 'Abaixo_Last_CG_A', 'CV_ML', 'CV_Over']]

In [21]:
cor = cor.corr()

In [23]:
cor['Back_Over']

Back_Over            1.000000
P(H)                -0.034153
P(A)                 0.028949
P(O)                 0.004177
P(U)                -0.018275
P_Diff              -0.033888
Porc_Over_Home      -0.004316
Porc_Over_Away      -0.028691
Last_CG_H            0.036758
Last_CG_A            0.009565
MediaCustoGolHome    0.008261
MediaCustoGolAway   -0.006936
Acima_Last_CG_H      0.009323
Acima_Last_CG_A     -0.010380
Abaixo_Last_CG_H    -0.007794
Abaixo_Last_CG_A    -0.005900
CV_ML                0.023242
CV_Over             -0.009710
Name: Back_Over, dtype: float64

In [ ]:
treino.to_excel('KBO-2019-2023.xlsx', index=False)

In [ ]:
treino.to_excel('~/Desktop/MLB-2021-23.xlsx', index=False)

### Fatiando o DF

In [ ]:
def corta_df(df):
    corte = (df.P_Diff >= 0.03) & (df.P_Diff <= 0.1) & (df['P(O)'] >= 0.50) & (df['P(O)'] <= 0.7) & (df['Abaixo_Last_CG_H'] == 0) & (df['Porc_Over_Home'] >= 0) & (df['Porc_Over_Home'] <= 0.5)
    df = df[corte]

    return df

In [ ]:
cortado = corta_df(treino)
sns.scatterplot(data=cortado, x='Porc_Over_Home', y='MediaCustoGolHome', hue='PL')

### Testando modelos em diferentes seeds

In [ ]:
treino.columns

In [ ]:
features = ['Odds_H', 'P_Diff', 'Odds_Over', 'Porc_Over_Home']

In [ ]:
modelos = ['knn', 'gbc', 'ada', 'lightgbm', 'nb', 'qda', 'rf', 'et', 'dt', 'rbfsvm', 'gpc', 'mlp']
best_list = []
pos = 0

teste = pd.read_csv('../../data/MLB/mlb-2022.csv')
teste = prepara_df(teste)
teste = corta_df(teste)

print(f'{len(teste)} partidas na temporada')

lista = random.sample(range(1, 9999), 1) # gera números entre 1 e 9999

for i in modelos:
  print('-------------------------------------------')
  print(f'Modelo {i}')
  print('')
  melhor_seed = 0
  melhor_alvo = 0

  for r in range(len(lista)):
    resultados = pd.DataFrame()
    
    classificador = setup(data = cortado, target = target, session_id=lista[r], ignore_features = [x for x in cortado.columns.to_list() if x not in features and x != target], verbose=False)
    Treino = create_model(i, verbose=False)
    session_id = get_config('seed')

    df1 = predict_model(Treino, teste, verbose=False)

    df1 = calcula_profit(df1)

    filtro = df1.prediction_label == 1
    df0 = df1[filtro]

    # Ajustando o Índice
    df0.reset_index(inplace=True, drop=True)
    # df0.index = df0.index.set_names(['Nº'])
    df0 = df0.rename(index=lambda x: x + 1)

    df0['Profit_acu'] = df0.Profit.cumsum()
    
    profit = df0.Profit_acu.tail(1).item()
    ROI = (df0.Profit_acu.tail(1)/len(df0)*100).item()
    resultados.loc[resultados.shape[0],['Modelo','Profit','ROI','Entradas']] = [i, profit, ROI, len(df0)]

    resultados['Profit'] = resultados['Profit'].replace('', np.nan)
    resultados['ROI'] = resultados['ROI'].replace('', np.nan)
    PL_total = round(resultados['Profit'].astype(float).sum(), 2)
    ROI_total = round((PL_total/ resultados['Entradas'].sum())*100, 2)
    winrate = calcula_wr(df0)
    odd_media = df0.Odds_Over.mean()
    odd_media = round(odd_media, 2)
    print('')
    print(f'WR: {winrate}% | Odd média: {odd_media:.2f}')
    print(f'PL: {PL_total}u')
    print(f'ROI: {ROI_total}% em {len(df0)} jogos')
    print(f'Seed: {session_id}')
    std_total = round(resultados['ROI'].std(), 2)

    if PL_total < 0:
      alvo = PL_total * ROI_total * -1
    else: alvo = PL_total * ROI_total

    if ROI_total >= 3:
      best_list.append(Treino)
      print(f'Add na posição {pos}')
      pos += 1

    if alvo > melhor_alvo:
      melhor_alvo = alvo
      melhor_seed = session_id
      print('**NEW BEST**')

  print('')
  print(melhor_seed)

In [ ]:
len(best_list)

### Testando em vários anos

In [ ]:
lucros = []
anos = [2022, 2023]

for indice, model in enumerate(best_list):
    print(f'{nome_modelo} - {indice}')
    print('')
    df_total = pd.DataFrame()
    for ano in anos:
        teste = pd.read_csv(f'../data/mlb-{ano}.csv')
        teste = prepara_df(teste)
        teste = corta_df(teste)

        df1 = predict_model(model, teste, verbose=False)
        df1 = calcula_profit(df1)

        filtro = df1.prediction_label == 1
        df0 = df1[filtro]

        # Ajustando o Índice
        df0.reset_index(inplace=True, drop=True)
        df0 = df0.rename(index=lambda x: x + 1)

        df0['Profit_acu'] = df0.Profit.cumsum()
    	
        nome_modelo = type(best_list[indice]).__name__
        winrate = calcula_wr(df0)
        profit = round(df0.Profit_acu.tail(1).item(),2)
        ROI = round((df0.Profit_acu.tail(1)/len(df0)*100).item(),2)
        print(ano)
        print(f'WR: {winrate}% | Odd média: {odd_media:.2f}')
        print("Profit:",profit,"stakes em", len(df0),"jogos")
        print("ROI:",ROI,"%")
        print('---------------------------------------------')

        df_total = pd.concat([df_total, df0])
        lucros.append(profit)

    df_total.reset_index(inplace=True)
    df_total.sort_values('Date')

    df_total['Profit_acu'] = df_total.Profit.cumsum()

    profit = round(df_total.Profit_acu.tail(1).item(),2)
    ROI = round((df_total.Profit_acu.tail(1)/len(df_total)*100).item(),2)
    print("Total")
    print("Profit:",profit,"stakes em", len(df_total),"jogos")
    print("ROI:",ROI,"%")
    print('')
    int_confianca(df_total)
    print('')
    teste_variancia(df_total)
    plota_grafico(df_total)
    print('')
    print('')
    print('')
    print('')

### Dissecando o modelo

In [ ]:
modelo = best_list[1]
modelo

In [ ]:
sns.scatterplot(data=df_total, x='P(O)', y='P_Diff', hue='PL')

In [ ]:
teco = load_model('MLB_over', verbose=False)

In [ ]:
lucros = []
anos = [2022, 2023]

print('')
df_total = pd.DataFrame()
for ano in anos:
    teste = pd.read_csv(f'../data/mlb-{ano}.csv')
    teste = prepara_df(teste)
    teste['CV'] = (teste[['Odds_H', 'Odds_A']].std(axis=1)) / (teste[['Odds_H', 'Odds_A']].mean(axis=1))
    # teste = corta_df(teste)
    corte = ((teste['Over_Line'] <= 8.5) & (teste['CV'] > 0.05 ) & (teste['CV'] < 0.25 ))
    corte = ((teste['Over_Line'] <= 8.5) & ((teste['CV'] > 0.05 ) & (teste['CV'] < 0.25 ) | (teste['CV'] > 0.35 )))
    teste = teste[corte]

    df1 = predict_model(teco, teste, verbose=False)
    df1 = calcula_profit(df1)

    filtro = df1.prediction_label == 1
    df0 = df1[filtro]

    # Ajustando o Índice
    df0.reset_index(inplace=True, drop=True)
    df0 = df0.rename(index=lambda x: x + 1)

    df0['Profit_acu'] = df0.Profit.cumsum()
    
    winrate = calcula_wr(df0)
    profit = round(df0.Profit_acu.tail(1).item(),2)
    ROI = round((df0.Profit_acu.tail(1)/len(df0)*100).item(),2)
    odd_media = df0['Odds_Over'].mean()
    print(ano)
    print(f'WR: {winrate}% | Odd média: {odd_media:.2f}')
    print("Profit:",profit,"stakes em", len(df0),"jogos")
    print("ROI:",ROI,"%")
    print('---------------------------------------------')

    df_total = pd.concat([df_total, df0])
    lucros.append(profit)

df_total.reset_index(inplace=True)
df_total.sort_values('Date')

df_total['Profit_acu'] = df_total.Profit.cumsum()

profit = round(df_total.Profit_acu.tail(1).item(),2)
ROI = round((df_total.Profit_acu.tail(1)/len(df_total)*100).item(),2)
print("Total")
print("Profit:",profit,"stakes em", len(df_total),"jogos")
print("ROI:",ROI,"%")
print('')
plota_grafico(df_total)

In [ ]:
risco_ruina(df_total, stake= 3, num_blocos=10)

### Salvando o modelo

In [ ]:
with open('serie_b_back_home.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)

### Teste de modelo pronto .pkl

In [ ]:
temp = 2022

teste = prepara_teste(temp)
teste = teste[teste.FT_Odds_H < teste.FT_Odds_A]

# Carregando o modelo
with open('serie_a_back_home.pkl', 'rb') as file:
    modelo = pickle.load(file)

y = teste.loc[:,'Back_Home']

# Criando o df só com as variáveis para o modelo
features_in = modelo.feature_names_in_
teste1 = teste.loc[:, features_in]

# Fazendo as predicoes do dia
predicoes = modelo.predict(teste1)

# Adicionando as previsões ao df completo
teste['Back_Home'] = y
teste['prediction_label'] = predicoes

teste = calcula_profit(teste)

filtro = teste.prediction_label == 1
df0 = teste[filtro]

# Ajustando o Índice
df0.reset_index(inplace=True, drop=True)
df0.index = df0.index.set_names(['Nº'])
df0 = df0.rename(index=lambda x: x + 1)

df0['Profit_acu'] = df0.Profit.cumsum()

lucro_acumulado_mes_a_mes = df0.groupby('mes')['Profit_acu'].last().tolist()
last_x = teste.Lucro_Acumulado.index[-1]
last_y = teste.Lucro_Acumulado.values[-1]
plt.text(last_x, last_y, str(round(last_y, 2)))
plt.show()

profit = round(df0.Profit_acu.tail(1).item(),2)
ROI = round((df0.Profit_acu.tail(1)/len(df0)*100).item(),2)
print(f'Temporada: {temp} ({len(teste)} jogos)')
print("Profit:",profit,"stakes em", len(df0),"jogos")
print("ROI:",ROI,"%")
print('')

int_confianca(df0)